In [ ]:
!pip install torch torchvision numpy matplotlib tqdm
!pip install pycocotools

In [ ]:
# Download and extract the COCO 2017 dataset (test and validation)
!mkdir -p /content/data/coco
!cd /content/data/coco && wget http://images.cocodataset.org/zips/val2017.zip && unzip val2017.zip
!cd /content/data/coco && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip && unzip annotations_trainval2017.zip

--2023-12-29 21:15:20--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.28.110, 3.5.25.69, 52.217.128.249, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.28.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip.1’

val2017.zip.1       100%[===================>] 777.80M  13.4MB/s    in 72s     

2023-12-29 21:16:33 (10.8 MB/s) - ‘val2017.zip.1’ saved [815585330/815585330]

Archive:  val2017.zip
replace val2017/000000212226.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: --2023-12-29 21:20:50--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.3.189, 54.231.231.113, 16.182.99.217, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.3.189|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (24

In [ ]:
from tqdm.auto import tqdm

In [ ]:
class_ids = [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21,
             22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
             42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
             60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80,
             81, 82, 84, 85, 86, 87, 88, 89, 90]

class_names = ["_background_","person", "bicycle", "car", "motorcycle", "airplane", "bus", "train",
               "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter",
               "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear",
               "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
               "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
               "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork",
               "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant",
               "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard",
               "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
               "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

In [ ]:
len(class_names)
len(class_ids)

81

Load data


In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16,SSD300_VGG16_Weights,retinanet_resnet50_fpn,RetinaNet_ResNet50_FPN_Weights,fasterrcnn_resnet50_fpn,FasterRCNN_ResNet50_FPN_Weights
from torchvision.transforms import functional as F
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.datasets import CocoDetection, VOCDetection
from tqdm.auto import tqdm

data = CocoDetection("/content/data/coco/val2017", "/content/data/coco/annotations/instances_val2017.json")

loading annotations into memory...
Done (t=0.79s)
creating index...
index created!


In [ ]:
voc_data = VOCDetection(root='/content/voc', download=True, image_set="val")


Using downloaded and verified file: /content/voc/VOCtrainval_11-May-2012.tar
Extracting /content/voc/VOCtrainval_11-May-2012.tar to /content/voc


In [ ]:
import json
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

def evaluate_coco_detection(annFile, resFile, annType='bbox'):
    # Load COCO dataset
    coco = COCO(annFile)

    # Load COCO detections
    cocoDt = coco.loadRes(resFile)

    # Get image IDs
    imgIds = sorted(coco.getImgIds())

    # Initialize COCO evaluation
    cocoEval = COCOeval(coco, cocoDt, annType)
    cocoEval.params.imgIds = imgIds

    # Run evaluation
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()


In [ ]:
def generate_result(model, data, preprocess, precision=8):
    results = []

    for idx, (image, _) in tqdm(enumerate(data)):
        # Preprocess the image
        processed_image = preprocess(image).to(device)
        batch = [processed_image]

        # Get predictions from the model
        predictions = model(batch)[0]

        # Detach and move to CPU
        predictions['boxes'] = predictions['boxes'].detach().to('cpu')
        predictions['scores'] = predictions['scores'].detach().to('cpu')
        predictions['labels'] = predictions['labels'].detach().to('cpu')
        torch.cuda.empty_cache()

        # Process each prediction
        for i in range(len(predictions['boxes'])):
            result = {
                'image_id': data.ids[idx],
                'bbox': [
                    round(coord.item(), precision) for coord in predictions['boxes'][i]
                ],
                'category_id': predictions['labels'][i].item(),
                'score': predictions['scores'][i].item(),
            }

            # Adjust the bounding box coordinates
            result['bbox'][2] -= result['bbox'][0]
            result['bbox'][3] -= result['bbox'][1]

            results.append(result)

    return results


In [ ]:
weights_ssd = SSD300_VGG16_Weights.COCO_V1

model_ssd = ssd300_vgg16(weights=weights_ssd)
model_ssd.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_ssd = model_ssd.to(device)
torch.cuda.empty_cache()


ssd_results = generate_result(model_ssd, data, weights_ssd.transforms())

0it [00:00, ?it/s]

In [ ]:
import json

def save_detection_results_to_json(ssd_results, output_json_path):
    with open(output_json_path, 'w') as json_file:
        json.dump(ssd_results, json_file)

    print(f"Detection results saved to: {output_json_path}")

In [ ]:
output_json_path = '/content/detection_results_ssd.json'
save_detection_results_to_json(ssd_results, output_json_path)


annFile = '/content/data/coco/annotations/instances_val2017.json'
evaluate_coco_detection(annFile, output_json_path)

Detection results saved to: /content/detection_results_ssd.json
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!
Loading and preparing results...
DONE (t=8.50s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=69.35s).
Accumulating evaluation results...
DONE (t=16.83s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.251
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.415
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.262
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.055
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.435
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.239
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.344

In [ ]:
# Load Faster R-CNN model
weights_frcnn=FasterRCNN_ResNet50_FPN_Weights.COCO_V1
model_frcnn = fasterrcnn_resnet50_fpn(weights=weights_frcnn)
model_frcnn.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_frcnn = model_frcnn.to(device)
torch.cuda.empty_cache()

# Generate results using Faster R-CNN
frcnn_results = generate_result(model_frcnn, data, weights_frcnn.transforms())

0it [00:00, ?it/s]

In [ ]:
output_json_path = '/content/detection_results_faster_rcnn.json'
save_detection_results_to_json(frcnn_results, output_json_path)

annFile = '/content/data/coco/annotations/instances_val2017.json'
evaluate_coco_detection(annFile, output_json_path)


Detection results saved to: /content/detection_results_faster_rcnn.json
loading annotations into memory...
Done (t=0.40s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.62s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=30.01s).
Accumulating evaluation results...
DONE (t=7.48s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.370
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.586
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.398
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.308
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] 

In [ ]:
# Load RetinaNet model
weights_retinanet = RetinaNet_ResNet50_FPN_Weights.COCO_V1
model_retinanet = retinanet_resnet50_fpn(weights=weights_retinanet)
model_retinanet.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_retinanet = model_retinanet.to(device)
torch.cuda.empty_cache()

# Generate results using RetinaNet
retinanet_results = generate_result(model_retinanet, data, weights_retinanet.transforms())


0it [00:00, ?it/s]

In [ ]:
output_json_path = '/content/detection_results_ratina_net.json'
save_detection_results_to_json(retinanet_results, output_json_path)


annFile = '/content/data/coco/annotations/instances_val2017.json'
evaluate_coco_detection(annFile, output_json_path)


Detection results saved to: /content/detection_results_ratina_net.json
loading annotations into memory...
Done (t=1.30s)
creating index...
index created!
Loading and preparing results...
DONE (t=6.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=62.11s).
Accumulating evaluation results...
DONE (t=17.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.363
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.557
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.191
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.489
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.314
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] 